# Libraries

In [53]:
import pandas as pd
import numpy as np
import pickle

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Data Loading/Reading

In [54]:
df = pd.read_csv("data.csv")
df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,5/2/2014 0:00,3.130000e+05,3.0,1.50,1340.0,7912.0,1.5,0.0,0.0,3,1340.0,0.0,1955.0,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,5/2/2014 0:00,2.384000e+06,5.0,2.50,3650.0,9050.0,2.0,0.0,4.0,5,3370.0,280.0,1921.0,0,709 W Blaine St,Seattle,WA 98119,USA
2,5/2/2014 0:00,3.420000e+05,3.0,2.00,1930.0,11947.0,1.0,0.0,0.0,4,1930.0,0.0,1966.0,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,5/2/2014 0:00,4.200000e+05,3.0,2.25,2000.0,8030.0,1.0,0.0,0.0,4,1000.0,1000.0,1963.0,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,5/2/2014 0:00,5.500000e+05,4.0,2.50,1940.0,10500.0,1.0,0.0,0.0,4,1140.0,800.0,1976.0,1992,9105 170th Ave NE,Redmond,WA 98052,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,7/9/2014 0:00,3.081667e+05,3.0,1.75,1510.0,6360.0,1.0,0.0,0.0,4,1510.0,0.0,1954.0,1979,501 N 143rd St,Seattle,WA 98133,USA
4596,7/9/2014 0:00,5.343333e+05,3.0,NaN,1460.0,7573.0,2.0,0.0,0.0,3,1460.0,0.0,1983.0,2009,14855 SE 10th Pl,Bellevue,WA 98007,USA
4597,7/9/2014 0:00,4.169042e+05,3.0,NaN,3010.0,7014.0,2.0,0.0,0.0,3,3010.0,0.0,2009.0,0,759 Ilwaco Pl NE,Renton,WA 98059,USA
4598,7/10/2014 0:00,2.034000e+05,4.0,2.00,2090.0,6630.0,1.0,0.0,0.0,3,1070.0,1020.0,1974.0,0,5148 S Creston St,Seattle,WA 98178,USA


# Data Exploration

In [55]:
print(f"Number of Rows: {df.shape[0]} \nNumber of Columns: {df.shape[1]}")

Number of Rows: 4600 
Number of Columns: 18


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4597 non-null   float64
 2   bedrooms       4595 non-null   float64
 3   bathrooms      4598 non-null   float64
 4   sqft_living    4598 non-null   float64
 5   sqft_lot       4598 non-null   float64
 6   floors         4598 non-null   float64
 7   waterfront     4595 non-null   float64
 8   view           4598 non-null   float64
 9   condition      4600 non-null   int64  
 10  sqft_above     4597 non-null   float64
 11  sqft_basement  4597 non-null   float64
 12  yr_built       4594 non-null   float64
 13  yr_renovated   4600 non-null   int64  
 14  street         4597 non-null   object 
 15  city           4596 non-null   object 
 16  statezip       4597 non-null   object 
 17  country        4594 non-null   object 
dtypes: float

In [57]:
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
count,4.597000e+03,4595.000000,4598.000000,4598.000000,4.598000e+03,4598.000000,4595.000000,4598.000000,4600.000000,4597.000000,4597.000000,4594.000000,4600.000000
mean,5.521190e+05,3.401088,2.160668,2139.498913,1.485607e+04,1.511853,0.007182,0.240757,3.451739,1827.509463,312.122036,1970.799086,808.608261
std,5.639834e+05,0.908983,0.783920,963.387024,3.589177e+04,0.537905,0.084449,0.778558,0.677230,862.138540,464.228562,29.740666,979.414536
min,0.000000e+00,0.000000,0.000000,370.000000,6.380000e+02,1.000000,0.000000,0.000000,1.000000,370.000000,0.000000,1900.000000,0.000000
25%,3.238333e+05,3.000000,1.750000,1460.000000,5.001000e+03,1.000000,0.000000,0.000000,3.000000,1190.000000,0.000000,1951.000000,0.000000
50%,4.610000e+05,3.000000,2.250000,1980.000000,7.683000e+03,1.500000,0.000000,0.000000,3.000000,1590.000000,0.000000,1976.000000,0.000000
75%,6.550000e+05,4.000000,2.500000,2620.000000,1.100375e+04,2.000000,0.000000,0.000000,4.000000,2300.000000,610.000000,1997.000000,1999.000000
max,2.659000e+07,9.000000,8.000000,13540.000000,1.074218e+06,3.500000,1.000000,4.000000,5.000000,9410.000000,4820.000000,2014.000000,2014.000000


In [58]:
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,5/2/2014 0:00,313000.0,3.0,1.50,1340.0,7912.0,1.5,0.0,0.0,3,1340.0,0.0,1955.0,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,5/2/2014 0:00,2384000.0,5.0,2.50,3650.0,9050.0,2.0,0.0,4.0,5,3370.0,280.0,1921.0,0,709 W Blaine St,Seattle,WA 98119,USA
2,5/2/2014 0:00,342000.0,3.0,2.00,1930.0,11947.0,1.0,0.0,0.0,4,1930.0,0.0,1966.0,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,5/2/2014 0:00,420000.0,3.0,2.25,2000.0,8030.0,1.0,0.0,0.0,4,1000.0,1000.0,1963.0,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,5/2/2014 0:00,550000.0,4.0,2.50,1940.0,10500.0,1.0,0.0,0.0,4,1140.0,800.0,1976.0,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [59]:
df.tail()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
4595,7/9/2014 0:00,308166.6667,3.0,1.75,1510.0,6360.0,1.0,0.0,0.0,4,1510.0,0.0,1954.0,1979,501 N 143rd St,Seattle,WA 98133,USA
4596,7/9/2014 0:00,534333.3333,3.0,NaN,1460.0,7573.0,2.0,0.0,0.0,3,1460.0,0.0,1983.0,2009,14855 SE 10th Pl,Bellevue,WA 98007,USA
4597,7/9/2014 0:00,416904.1667,3.0,NaN,3010.0,7014.0,2.0,0.0,0.0,3,3010.0,0.0,2009.0,0,759 Ilwaco Pl NE,Renton,WA 98059,USA
4598,7/10/2014 0:00,203400.0000,4.0,2.00,2090.0,6630.0,1.0,0.0,0.0,3,1070.0,1020.0,1974.0,0,5148 S Creston St,Seattle,WA 98178,USA
4599,7/10/2014 0:00,220600.0000,3.0,2.50,1490.0,8102.0,2.0,0.0,0.0,4,1490.0,0.0,1990.0,0,18717 SE 258th St,Covington,WA 98042,USA


In [60]:
print("-- Attributes in Data --")
for cols in df.columns:
    print(cols)

-- Attributes in Data --
date
price
bedrooms
bathrooms
sqft_living
sqft_lot
floors
waterfront
view
condition
sqft_above
sqft_basement
yr_built
yr_renovated
street
city
statezip
country


count

In [61]:
df.count()

date             4600
price            4597
bedrooms         4595
bathrooms        4598
sqft_living      4598
sqft_lot         4598
floors           4598
waterfront       4595
view             4598
condition        4600
sqft_above       4597
sqft_basement    4597
yr_built         4594
yr_renovated     4600
street           4597
city             4596
statezip         4597
country          4594
dtype: int64

In [62]:
df['city'].unique()

array(['Shoreline', 'Seattle', 'Kent', 'Bellevue', 'Redmond',
       'Maple Valley', 'North Bend', 'Lake Forest Park', 'Sammamish',
       'Auburn', 'Des Moines', 'Bothell', 'Federal Way', 'Kirkland',
       'Issaquah', 'Woodinville', 'Normandy Park', 'Fall City', 'Renton',
       'Carnation', 'Snoqualmie', 'Duvall', 'Burien', 'Covington',
       'Inglewood-Finn Hill', 'Kenmore', 'Newcastle', 'Mercer Island',
       'Black Diamond', 'Ravensdale', 'Clyde Hill', 'Algona', 'Skykomish',
       'Tukwila', 'Vashon', 'Yarrow Point', 'SeaTac', 'Medina',
       'Enumclaw', 'Snoqualmie Pass', 'Pacific', 'Beaux Arts Village',
       'Preston', 'Milton', nan], dtype=object)

In [63]:
df.nunique()

date               70
price            1738
bedrooms           10
bathrooms          26
sqft_living       566
sqft_lot         3112
floors              6
waterfront          2
view                5
condition           5
sqft_above        511
sqft_basement     207
yr_built          115
yr_renovated       60
street           4522
city               44
statezip           77
country             1
dtype: int64

Null

In [64]:
df.isnull().sum()

date             0
price            3
bedrooms         5
bathrooms        2
sqft_living      2
sqft_lot         2
floors           2
waterfront       5
view             2
condition        0
sqft_above       3
sqft_basement    3
yr_built         6
yr_renovated     0
street           3
city             4
statezip         3
country          6
dtype: int64

# Data Pre-Processing

In [65]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4597 non-null   float64
 2   bedrooms       4595 non-null   float64
 3   bathrooms      4598 non-null   float64
 4   sqft_living    4598 non-null   float64
 5   sqft_lot       4598 non-null   float64
 6   floors         4598 non-null   float64
 7   waterfront     4595 non-null   float64
 8   view           4598 non-null   float64
 9   condition      4600 non-null   int64  
 10  sqft_above     4597 non-null   float64
 11  sqft_basement  4597 non-null   float64
 12  yr_built       4594 non-null   float64
 13  yr_renovated   4600 non-null   int64  
 14  street         4597 non-null   object 
 15  city           4596 non-null   object 
 16  statezip       4597 non-null   object 
 17  country        4594 non-null   object 
dtypes: float

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
count,4.597000e+03,4595.000000,4598.000000,4598.000000,4.598000e+03,4598.000000,4595.000000,4598.000000,4600.000000,4597.000000,4597.000000,4594.000000,4600.000000
mean,5.521190e+05,3.401088,2.160668,2139.498913,1.485607e+04,1.511853,0.007182,0.240757,3.451739,1827.509463,312.122036,1970.799086,808.608261
std,5.639834e+05,0.908983,0.783920,963.387024,3.589177e+04,0.537905,0.084449,0.778558,0.677230,862.138540,464.228562,29.740666,979.414536
min,0.000000e+00,0.000000,0.000000,370.000000,6.380000e+02,1.000000,0.000000,0.000000,1.000000,370.000000,0.000000,1900.000000,0.000000
25%,3.238333e+05,3.000000,1.750000,1460.000000,5.001000e+03,1.000000,0.000000,0.000000,3.000000,1190.000000,0.000000,1951.000000,0.000000
50%,4.610000e+05,3.000000,2.250000,1980.000000,7.683000e+03,1.500000,0.000000,0.000000,3.000000,1590.000000,0.000000,1976.000000,0.000000
75%,6.550000e+05,4.000000,2.500000,2620.000000,1.100375e+04,2.000000,0.000000,0.000000,4.000000,2300.000000,610.000000,1997.000000,1999.000000
max,2.659000e+07,9.000000,8.000000,13540.000000,1.074218e+06,3.500000,1.000000,4.000000,5.000000,9410.000000,4820.000000,2014.000000,2014.000000


# Filling

In [66]:
def fillNaObject(cols):
    for i in cols:
        df[i] = df[i].fillna(df[i].mode()[0])

columns = ['city','street','country','statezip']
fillNaObject(columns)

In [67]:
def fillNaint(cols):
    for i in cols:
        df[i] = df[i].fillna(df[i].mode()[0])

columns = ['bathrooms','bedrooms','floors','waterfront','view','yr_built']
fillNaint(columns)

In [68]:
def fillNafloat(cols):
    for i in cols:
        df[i] = df[i].fillna(df[i].mean())

columns = ['price','sqft_living','sqft_lot','sqft_above','sqft_basement']
fillNafloat(columns)

In [69]:
def convertfloattoint(cols):
    for i in cols:
        df[i] = df[i].astype('int64')

columns = ['price','bedrooms','bathrooms','floors','waterfront','view','yr_built','sqft_living','sqft_lot','sqft_above','sqft_basement']
convertfloattoint(columns)

# 
Dropping useless 

In [70]:
df.drop('street', axis=1, inplace=True)

In [71]:
df.drop('country', axis=1, inplace=True)

In [72]:
df.drop('date', axis=1, inplace=True)

In [73]:
df.head(2)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city,statezip
0,313000,3,1,1340,7912,1,0,0,3,1340,0,1955,2005,Shoreline,WA 98133
1,2384000,5,2,3650,9050,2,0,4,5,3370,280,1921,0,Seattle,WA 98119


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   price          4600 non-null   int64 
 1   bedrooms       4600 non-null   int64 
 2   bathrooms      4600 non-null   int64 
 3   sqft_living    4600 non-null   int64 
 4   sqft_lot       4600 non-null   int64 
 5   floors         4600 non-null   int64 
 6   waterfront     4600 non-null   int64 
 7   view           4600 non-null   int64 
 8   condition      4600 non-null   int64 
 9   sqft_above     4600 non-null   int64 
 10  sqft_basement  4600 non-null   int64 
 11  yr_built       4600 non-null   int64 
 12  yr_renovated   4600 non-null   int64 
 13  city           4600 non-null   object
 14  statezip       4600 non-null   object
dtypes: int64(13), object(2)
memory usage: 539.2+ KB


In [75]:
def convertObjtoint(cols):
    for i in cols:
        datalabelEncoder = LabelEncoder()
        df[i] = datalabelEncoder.fit_transform(df[i])

columns = ['city', 'statezip']
convertObjtoint(columns)

In [76]:
df.corr()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city,statezip
price,1.000000,0.200582,0.317247,0.430468,0.050378,0.138174,0.135633,0.228433,0.034821,0.367732,0.210427,0.021375,-0.028805,0.018196,-0.043560
bedrooms,0.200582,1.000000,0.498250,0.594468,0.068773,0.156846,-0.006299,0.109904,0.025247,0.484087,0.334267,0.140706,-0.060414,-0.130325,-0.151660
bathrooms,0.317247,0.498250,1.000000,0.711241,0.109418,0.463866,0.071806,0.203389,-0.118819,0.648763,0.270245,0.394102,-0.185970,-0.069443,-0.153717
sqft_living,0.430468,0.594468,0.711241,1.000000,0.210505,0.340522,0.117606,0.310970,-0.062847,0.876222,0.447278,0.286676,-0.122747,-0.110045,-0.198958
sqft_lot,0.050378,0.068773,0.109418,0.210505,1.000000,-0.005274,0.017233,0.073877,0.000638,0.216439,0.034817,0.050096,-0.022812,-0.079222,-0.128562
floors,0.138174,0.156846,0.463866,0.340522,-0.005274,1.000000,0.008698,0.017326,-0.306175,0.516337,-0.253079,0.561487,-0.246690,0.035660,-0.086886
waterfront,0.135633,-0.006299,0.071806,0.117606,0.017233,0.008698,1.000000,0.360935,0.000352,0.078916,0.097506,-0.023718,0.008625,0.001424,0.007894
view,0.228433,0.109904,0.203389,0.310970,0.073877,0.017326,0.360935,1.000000,0.063077,0.174302,0.321617,-0.064607,0.022967,0.001245,0.079649
condition,0.034821,0.025247,-0.118819,-0.062847,0.000638,-0.306175,0.000352,0.063077,1.000000,-0.177736,0.200729,-0.397177,-0.186818,-0.010797,0.027528
sqft_above,0.367732,0.484087,0.648763,0.876222,0.216439,0.516337,0.078916,0.174302,-0.177736,1.000000,-0.038289,0.407206,-0.160195,-0.121881,-0.247570


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   price          4600 non-null   int64
 1   bedrooms       4600 non-null   int64
 2   bathrooms      4600 non-null   int64
 3   sqft_living    4600 non-null   int64
 4   sqft_lot       4600 non-null   int64
 5   floors         4600 non-null   int64
 6   waterfront     4600 non-null   int64
 7   view           4600 non-null   int64
 8   condition      4600 non-null   int64
 9   sqft_above     4600 non-null   int64
 10  sqft_basement  4600 non-null   int64
 11  yr_built       4600 non-null   int64
 12  yr_renovated   4600 non-null   int64
 13  city           4600 non-null   int64
 14  statezip       4600 non-null   int64
dtypes: int64(15)
memory usage: 539.2 KB


In [78]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
df1 = pd.DataFrame(scaled_data, columns=df.columns)

In [79]:
df1.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city,statezip
0,-0.424167,-0.441011,-1.048942,-0.830154,-0.193536,-0.831411,-0.085004,-0.309194,-0.667112,-0.565711,-0.672638,-0.532691,1.221670,0.861479,1.111597
1,3.249524,1.760453,0.280662,1.568419,-0.161819,0.981144,-0.085004,4.830079,2.286416,1.789923,-0.069224,-1.675734,-0.825693,0.778018,0.920314
2,-0.372724,-0.441011,0.280662,-0.217532,-0.081078,-0.831411,-0.085004,-0.309194,0.809652,0.118932,-0.672638,-0.162884,-0.825693,-0.640816,-0.609956
3,-0.234362,-0.441011,0.280662,-0.144848,-0.190247,-0.831411,-0.085004,-0.309194,0.809652,-0.960251,1.482411,-0.263740,-0.825693,-1.892728,-1.518553
4,-0.003759,0.659721,0.280662,-0.207148,-0.121406,-0.831411,-0.085004,-0.309194,0.809652,-0.797793,1.051401,0.173305,1.208396,0.444175,-0.370851


In [80]:
X = df1.drop(columns=['price','statezip'], axis=1)
y = df1['price']

# Train-Test Split

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [82]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3680, 13), (920, 13), (3680,), (920,))

# Model Training using SVC

In [83]:
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score

In [84]:
print("training using SVC ")
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)


training using SVC 


LinearRegression()

# Saving 

In [85]:
pickle.dump(model_reg, open('model_reg.pkl', 'wb'))

In [86]:
model = pickle.load(open('model_reg.pkl', 'rb'))

In [87]:
model_predictions = model_reg.predict(X_test)
mse = mean_squared_error(y_test, model_predictions)
r2 = r2_score(y_test, model_predictions)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

Mean Squared Error: 3.104369476883961
R2 Score: 0.032627878901952845
